<h1 style='font-size: 45px; color: Tomato; font-family: Candara; font-weight: 700'>Sentiment Classification Machine Learning Approach</h1>


<div style="text-align: center;">
    <img src="Resources/BBT.png", alt='BBT' width='600', height='450'
</div>

<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 700'>Import Required Libraries</h3>

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import joblib
import math

print('Libraries loaded Succesfully')

Libraries loaded Succesfully


<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 700'>Import and Preprocessing Dataset</h3>

In [18]:
df = pd.read_csv("Datasets/Building a Better Tomorrow (BBT) Initiative Sentiments.csv")
pd.set_option('display.max_colwidth', 120)
df.sample(10)

,Review_Date,Region,Reviews,Processed_Reviews,sentiment_score,sentiment_label
52014,2024-10-16,Mwanza,"Thanks to BBT, I now sell my produce directly to buyers without middlemen.",thank bbt sell produc directli buyer without middleman,0.3612,Positive
3345,2025-09-28,Mwanza,"The BBT grant allowed me to buy high-quality seeds, which increased my yield.",bbt grant allow buy highqual seed increas yield,0.5267,Positive
9152,2024-05-02,Dodoma,I’ve heard mixed reviews about the BBT initiative—some farmers benefit while others feel left out.,ive heard mix review bbt initiativesom farmer benefit other feel left,0.4588,Positive
31841,2019-06-05,Mwanza,Women in agriculture are finally being recognized and supported through BBT.,woman agricultur final recogn support bbt,0.4019,Positive
33557,2019-06-05,Tanga,"I support BBT, but I think Tanzania should also invest in technology and manufacturing.",support bbt think tanzania also invest technolog manufactur,0.4019,Positive
20266,2022-07-01,Arusha,The market access that BBT provides is game-changing for smallholder farmers in my area.,market access bbt provid gamechang smallhold farmer area,0.0000,Neutral
16681,2025-11-08,Tanga,Why does BBT focus only on crop farming? What about livestock farmers?,bbt focu crop farm livestock farmer,0.0000,Neutral
59613,2024-08-29,Arusha,"Agriculture is Tanzania’s backbone, and BBT is ensuring that the youth take an active role in its development.",agricultur tanzania backbon bbt ensur youth take activ role develop,0.0000,Neutral
59946,2023-08-09,Mwanza,"I’m happy to see BBT promoting soil conservation. It’s important for future generations, and I’ve started using bett...",im happi see bbt promot soil conserv import futur gener ive start use better techniqu protect land,0.6705,Positive
63921,2024-11-08,Mbeya,"BBT is promising, but it needs to be more inclusive to reach all farmers.",bbt promis need inclus reach farmer,0.0258,Neutral


<h4 style='font-size: 25px; color: Green; font-family: Candara; font-weight: 600'>1.1: Dataset Informations/ Overview</h4>

In [19]:
df.columns

Index(['Review_Date', 'Region', 'Reviews', 'Processed_Reviews',
       'sentiment_score', 'sentiment_label'],
      dtype='object')

In [20]:
df.shape

(70146, 6)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70146 entries, 0 to 70145
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Review_Date        70146 non-null  object 
 1   Region             70146 non-null  object 
 2   Reviews            70146 non-null  object 
 3   Processed_Reviews  70146 non-null  object 
 4   sentiment_score    70146 non-null  float64
 5   sentiment_label    70146 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.2+ MB


In [22]:
df.dtypes

Review_Date           object
Region                object
Reviews               object
Processed_Reviews     object
sentiment_score      float64
sentiment_label       object
dtype: object

<h4 style='font-size: 25px; color: Green; font-family: Candara; font-weight: 600'>1.2: Handling Missing Values</h4>

In [23]:
df.isnull().sum()

Review_Date          0
Region               0
Reviews              0
Processed_Reviews    0
sentiment_score      0
sentiment_label      0
dtype: int64

In [24]:
df.ffill(inplace=True)

<h2 style='font-size: 35px; color: Tomato; font-family: Candara; font-weight: 600'>2.0: Statistic Description of The Datasets</h2>

<h4 style='font-size: 25px; color: Green; font-family: Candara; font-weight: 600'>2.1: Overall Statistics Distributions </h4>

In [25]:
df.describe()

,sentiment_score
count,70146.000000
mean,0.265303
std,0.329957
min,-0.765000
25%,0.000000
50%,0.361200
75%,0.458800
max,0.930000


<h4 style='font-size: 25px; color: green; font-family: Candara; font-weight: 600'>2.3: Distributions of Categorical Variables</h4>

In [26]:
categories = ["Region", "sentiment_label"]
total_length = 100

for category in categories:
    print(f"\n{' ' * 35}Count occurrences of each {category}:")
    counts = df[category].value_counts()
    proportions = df[category].value_counts(normalize=True)
    for value, count in counts.items():
        proportion = proportions[value]
        text_length = len(f"{category}: {value}")
        remaining_length = total_length - text_length - len(str(count)) - len(f"{proportion:.2%}") - 3
        print(f"{category}: {value} {'.' * remaining_length} {count} ({proportion:.2%})")


                                   Count occurrences of each Region:
Region: Mwanza ........................................................................ 14110 (20.12%)
Region: Dodoma ........................................................................ 14091 (20.09%)
Region: Arusha ........................................................................ 13991 (19.95%)
Region: Mbeya ......................................................................... 13986 (19.94%)
Region: Tanga ......................................................................... 13968 (19.91%)

                                   Count occurrences of each sentiment_label:
sentiment_label: Positive ............................................................. 40055 (57.10%)
sentiment_label: Neutral .............................................................. 25138 (35.84%)
sentiment_label: Negative ............................................................... 4953 (7.06%)


<h4 style='font-size: 25px; color: Green; font-family: Candara; font-weight: 600'>2.4: Distributions Comparisons of Categorical Variables Across Other Groups</h4>

In [27]:
categories = ["Region", "sentiment_label"]
group_column = 'Region'
total_length = 100

for category in categories:
    if category == group_column:
        continue

    print(f"\n{' ' * 35}Distribution of {category} across {group_column}:")
    grouped = df.groupby(group_column)[category].value_counts(normalize=False)  # Counts
    grouped_proportions = df.groupby(group_column)[category].value_counts(normalize=True)  # Proportions
    
    for group, values in grouped.groupby(level=0):  # Level 0 is the group_column
        print(f"\n{group_column}: {group}")
        for (g, value), count in values.items():
            proportion = grouped_proportions[(group, value)]
            text_length = len(f"{category}: {value}")
            remaining_length = total_length - text_length - len(str(count)) - len(f"{proportion:.2%}") - 3
            
            print(f"{category}: {value} {'.' * remaining_length} {count} ({proportion:.2%})")



                                   Distribution of sentiment_label across Region:

Region: Arusha
sentiment_label: Positive .............................................................. 7960 (56.89%)
sentiment_label: Neutral ............................................................... 5064 (36.19%)
sentiment_label: Negative ................................................................ 967 (6.91%)

Region: Dodoma
sentiment_label: Positive .............................................................. 8050 (57.13%)
sentiment_label: Neutral ............................................................... 5020 (35.63%)
sentiment_label: Negative ............................................................... 1021 (7.25%)

Region: Mbeya
sentiment_label: Positive .............................................................. 7897 (56.46%)
sentiment_label: Neutral ............................................................... 5101 (36.47%)
sentiment_label: Negative ....................

<h1 style='font-size: 35px; color: Tomato; font-family: Candara; font-weight: 600'>3.0: Machine Learning Model Development</h1>

<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 600'>3.1: Feture Selection and Preprocessing</h3>

In [45]:
# Separate features and target
X = df.drop(columns=['Review_Date', 'sentiment_label', 'Reviews', 'sentiment_score'])
y = df['sentiment_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)            # Split the data into training and testing sets
categorical_features = ['Processed_Reviews', 'Region']                                               # Define categorical and numerical features
preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])  # Preprocessing pipeline

<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 600'>3.2: Model Selection and Evaluation</h3>

In [47]:

# Define models to evaluate
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(kernel='linear', random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

# Train and evaluate models
results = []

for model_name, model in models.items():
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', model) ])        # Create a pipeline with preprocessing and the model
    pipeline.fit(X_train, y_train)       # Train the model
    y_pred = pipeline.predict(X_test)    # Predict on the test set
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Cross-validation score
    cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    
    # Store results
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'CV Accuracy': cv_mean
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
results_df

,Model,Accuracy,Precision,Recall,F1 Score,CV Accuracy
0,Random Forest,1.0,1.0,1.0,1.0,1.0
1,SVM,1.0,1.0,1.0,1.0,1.0
2,Logistic Regression,1.0,1.0,1.0,1.0,1.0


<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 600'>3.3: Saving The Best Model</h3>

In [49]:
# Save the best model
best_model_name = results_df.loc[results_df['Accuracy'].idxmax(), 'Model']
best_model = models[best_model_name]

# Retrain the best model on the full dataset
best_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', best_model)])
best_pipeline.fit(X, y)

# Save the best model to disk
joblib.dump(best_pipeline, f'{best_model_name}.pkl')
print(f"Best model ({best_model_name}) saved to '{best_model_name}.pkl'.")

Best model (Random Forest) saved to 'Random Forest.pkl'.


<h1 style='font-size: 35px; color: Tomato; font-family: Candara; font-weight: 600'>4.0: Prediction From New Data</h1>

<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 600'>4.1: Input Form for New Data</h3>

In [52]:
loaded_model = joblib.load(f'{best_model_name}.pkl')
def format_prompt(prompt_text):
    total_length = 60
    dots_length = total_length - len(prompt_text)
    return f"{prompt_text} {'.' * dots_length}"
def predict_bean_variety():
    while True:
        print("\nEnter the following details to predict the bean variety (or type 'exit' to quit):\n")
        try:
            # Define default values for each feature
            defaults = {
                "Region": "Arusha",
                "Processed_Reviews": "train provid bbt except learn busi strategi farm techniqu"
            }

            # Initialize input data dictionary
            input_data = {}

            # Gather inputs and handle empty inputs
            for field, default_value in defaults.items():
                user_input = input(format_prompt(f"{field} (e.g., {default_value}): "))
                input_data[field] = user_input if user_input else default_value

            # Convert input data to a DataFrame
            input_df = pd.DataFrame([input_data])

            # Use the loaded model to predict the bean variety
            Predicted_sentiment = loaded_model.predict(input_df)[0]

            # Display the predicted variety
            print(f"\nPredicted Sentiment: {Predicted_sentiment}")

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")

        # Ask the user if they want to predict again
        predict_again = input("\nDo you want to predict again? (yes/no): ").strip().lower()
        if predict_again != 'yes':
            print("Exiting the prediction tool. Goodbye!")
            break



In [53]:
# Run the prediction tool
predict_bean_variety()


Enter the following details to predict the bean variety (or type 'exit' to quit):



Region (e.g., Arusha):  ..................................... Arusha
Processed_Reviews (e.g., train provid bbt except learn busi strategi farm techniqu):   Good



Predicted Sentiment: Positive



Do you want to predict again? (yes/no):  Yes



Enter the following details to predict the bean variety (or type 'exit' to quit):



...


Enter the following details to predict the bean variety (or type 'exit' to quit):



Region (e.g., Arusha):  ..................................... BBT IS LYING AND CORRUPTION
Processed_Reviews (e.g., train provid bbt except learn busi strategi farm techniqu):   



Predicted Sentiment: Neutral



Do you want to predict again? (yes/no):  NO


Exiting the prediction tool. Goodbye!


<h3 style='font-size: 25px; color: Tomato; font-family: Candara; font-weight: 600'>4.2: Prediction Form File Data</h3>

In [54]:
loaded_model = joblib.load(f'{best_model_name}.pkl')

def predict_dataframe(df):
    try:
        required_columns = ['Region', 'Processed_Reviews']
        if not all(column in df.columns for column in required_columns):
            print("Error: The DataFrame is missing required columns.")
            return None
        
        predictions = loaded_model.predict(df)  # Make predictions
        df['Predicted_sentiment'] = predictions   # Add predictions to the DataFrame
        return df
    
    except Exception as e:
        print(f"An error occurred while processing the DataFrame: {e}")
        return None

# Example usage
if __name__ == "__main__":
    result_df = predict_dataframe(df)
    if result_df is not None:
        print("Predictions added to the DataFrame:")
        #print(result_df)

Predictions added to the DataFrame:


In [55]:
result_df[['Processed_Reviews', 'Processed_Reviews',  'sentiment_label', 'Predicted_sentiment']]

,Processed_Reviews,Processed_Reviews,sentiment_label,Predicted_sentiment
0,train provid bbt except learn busi strategi farm techniqu,train provid bbt except learn busi strategi farm techniqu,Neutral,Neutral
1,ive abl diversifi farm product support bbt instrument process,ive abl diversifi farm product support bbt instrument process,Positive,Positive
2,bbt potenti clear transpar process alloc fund,bbt potenti clear transpar process alloc fund,Positive,Positive
3,agricultur extens offic bbt regularli visit u ensur follow best practic,agricultur extens offic bbt regularli visit u ensur follow best practic,Positive,Positive
4,govern final given agricultur attent deserv,govern final given agricultur attent deserv,Neutral,Neutral
...,...,...,...,...
70141,transpar need select process bbt fund,transpar need select process bbt fund,Neutral,Neutral
70142,young peopl embrac farm secur tanzania agricultur futur,young peopl embrac farm secur tanzania agricultur futur,Neutral,Neutral
70143,bbt expand includ livestock farm well youth benefit,bbt expand includ livestock farm well youth benefit,Positive,Positive
70144,see potenti bbt still barrier prevent widespread success,see potenti bbt still barrier prevent widespread success,Positive,Positive


---
This analysis was performed by **Jabulente**, a passionate and dedicated data scientist with a strong commitment to using data to drive meaningful insights and solutions. For inquiries, collaborations, or further discussions, please feel free to reach out via email at **[Jabulente@hotmai.com](Jabulente@hotmai.com)**

----

<h1 style='font-size: 65px; color: crimson; font-family: Candara; font-weight: 600; text-align: center'>THE END</h1>